In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.1'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz

!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.0.1-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [2]:

from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StringType, DateType,IntegerType


# Create a SparkSession
spark = SparkSession.builder.appName("SkillDrill8-1").getOrCreate()
spark.conf.set("spark.sql.shuffle.partitions", 4) 

In [3]:
df = spark.read.csv("../Resources/airportCodes.csv", header=True, inferSchema=True) 
df=df.filter("country='USA'") 


In [4]:
df.show() 

+-----------------+-------+-----------+
|             City|country|airportCode|
+-----------------+-------+-----------+
|     Aberdeen, SD|    USA|        ABR|
|      Abilene, TX|    USA|        ABI|
|        Akron, OH|    USA|        CAK|
|      Alamosa, CO|    USA|        ALS|
|       Albany, GA|    USA|        ABY|
|       Albany, NY|    USA|        ALB|
|  Albuquerque, NM|    USA|        ABQ|
|   Alexandria, LA|    USA|        AEX|
|    Allentown, PA|    USA|        ABE|
|     Alliance, NE|    USA|        AIA|
|       Alpena, MI|    USA|        APN|
|      Altoona, PA|    USA|        AOO|
|     Amarillo, TX|    USA|        AMA|
|    Anchorage, AK|    USA|        ANC|
|     Appleton, WI|    USA|        ATW|
|    Asheville, NC|    USA|        AVL|
|        Aspen, CO|    USA|        ASE|
|       Athens, GA|    USA|        AHN|
|      Atlanta, GA|    USA|        ATL|
|Atlantic City, NJ|    USA|        ACY|
+-----------------+-------+-----------+
only showing top 20 rows



In [5]:
df.write.parquet('parquet_codes_basic',mode='overwrite') 

In [6]:
p_df=spark.read.parquet('parquet_codes_basic') 

In [7]:
p_df.createOrReplaceTempView('p_codes') 

In [8]:
df.write.partitionBy("City").mode("overwrite").parquet("delayed_partitioned") 

In [9]:
p_df_p=spark.read.parquet('delayed_partitioned') 

In [10]:
p_df_p.createOrReplaceTempView('p_codes_p') 

In [11]:
spark.sql("cache table p_codes_p") 

DataFrame[]

In [12]:
spark.catalog.isCached("p_codes_p") 

True

In [13]:
spark.sql("uncache table p_codes_p")


DataFrame[]

In [14]:
 #Verify that the table is no longer cached
if spark.catalog.isCached("p_codes_p") :
  print("a table is till cached")
else:
  print("all clear")

all clear
